In [1]:
from sklearn.linear_model import LinearRegression
import pandas as pd

df = pd.read_csv('World-Stock-Prices-Dataset.csv')


df["Country"].unique()


array(['usa', 'germany', 'japan', 'switzerland', 'canada', 'netherlands',
       'france'], dtype=object)

In [2]:
# Explore the dataset structure
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)
print("\nBasic statistics:")
print(df.describe())


Dataset shape: (310122, 13)

Column names:
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Brand_Name', 'Ticker', 'Industry_Tag', 'Country', 'Dividends', 'Stock Splits', 'Capital Gains']

First few rows:
                        Date        Open        High         Low       Close  \
0  2025-07-03 00:00:00-04:00    6.630000    6.740000    6.615000    6.640000   
1  2025-07-03 00:00:00-04:00  106.750000  108.370003  106.330101  107.339996   
2  2025-07-03 00:00:00-04:00  122.629997  123.050003  121.550003  121.930000   
3  2025-07-03 00:00:00-04:00  221.705002  224.009995  221.360001  223.410004   
4  2025-07-03 00:00:00-04:00  212.145004  214.649994  211.810104  213.550003   

       Volume Brand_Name Ticker Industry_Tag  Country  Dividends  \
0   4209664.0    peloton   PTON      fitness      usa        0.0   
1    560190.0      crocs   CROX     footwear      usa        0.0   
2     36600.0     adidas  ADDYY      apparel  germany        0.0   
3  29295154.0     amazon   AMZN   e-com

In [3]:
#questions 1-3
#1
top_row = df.loc[df["High"].idxmax()]
print(top_row)
#2
avg = df.groupby("Brand_Name")["Volume"].mean().to_dict()
print(avg)
#3
country_counts = df["Country"].value_counts().to_dict()
print(country_counts)

Date             2024-06-18 00:00:00-04:00
Open                                3435.0
High                           3463.070068
Low                                 3370.0
Close                          3427.610107
Volume                            693800.0
Brand_Name                        chipotle
Ticker                                 CMG
Industry_Tag                          food
Country                                usa
Dividends                              0.0
Stock Splits                           0.0
Capital Gains                          NaN
Name: 18924, dtype: object
{'3m': 3480991.8792890264, 'adidas': 59576.23362175525, 'adobe': 5115460.557032458, 'airbnb': 6007634.09325562, 'amazon': 116452264.35935085, 'amd': 34004310.76893354, 'american eagle outfitters': 5032047.939258114, 'american express': 6076273.512982998, 'apple': 376723829.8304482, 'block': 10403551.52184874, 'bmw group': 12802.4037999038, 'chipotle': 1386581.7429726997, 'cisco': 41715528.26846986, 'coinbase': 

4) which company had the most volatile stock in terms of daily range (high to low gap)

In [4]:
df['Volatility (%)'] = (df['High'] - df['Low']) / df['Low'] * 100
max_vol_row = df.loc[df['Volatility (%)'].idxmax()]
print(max_vol_row)

Date              2019-06-10 00:00:00-04:00
Open                             590.928222
High                             600.041075
Low                               67.861498
Close                             92.097748
Volume                                200.0
Brand_Name                             puma
Ticker                                PMMAF
Industry_Tag                        apparel
Country                             germany
Dividends                               0.0
Stock Splits                           10.0
Capital Gains                           NaN
Volatility (%)                   784.214303
Name: 94646, dtype: object


5) identify the companies that paid dividends at some point in the dataset

In [5]:
df[df["Dividends"] > 0]["Brand_Name"].unique()

array(['american express', 'cisco', 'jpmorgan chase & co',
       'the walt disney company', 'fedex', 'salesforce / slack',
       'southwest airlines', 'the coca-cola company', 'google',
       'the home depot', 'nike', "mcdonald's", 'johnson & johnson',
       'hilton', '3m', 'marriott', 'unilever', 'hershey company',
       'starbucks', 'microsoft', 'target', 'delta air lines', 'visa',
       'costco', 'procter & gamble', 'colgate palmolive',
       'american eagle outfitters', 'mastercard', 'nvidia', 'nordstrom',
       'apple', 'logitech', 'porsche', 'puma', 'adidas', 'bmw group',
       'lvmh', 'foot locker', 'honda', 'philips', 'toyota', 'ubisoft',
       'nintendo', 'adobe'], dtype=object)

6. which company's stock showed the greatest upward trend over time?

In [6]:
df['Date'] = pd.to_datetime(df['Date'], utc=True)
results = []

for company, group in df.groupby('Brand_Name'):
  group = group.sort_values(by='Date')
  if len(group) < 2:
    continue
  x = group['Date'].map(pd.Timestamp.toordinal).values.reshape(-1, 1)
  y = group['Close'].values

  model = LinearRegression().fit(x, y)

  start_date = x[0, 0]
  end_date = x[-1, 0]
  start_pred = model.predict([[start_date]])[0]
  end_pred = model.predict([[end_date]])[0]

  increase = (end_pred - start_pred) / start_pred * 100

  results.append({
    'Brand_Name': company,
    'Start_Date': group['Date'].iloc[0],
    'End_Date': group['Date'].iloc[-1],
    'Start_Price': start_pred,
    'End_Price': end_pred,
    'Increase': increase,
  })

results_df = pd.DataFrame(results)

max_increase = results_df.loc[results_df['Increase'].idxmax()]
print(max_increase)

Brand_Name                      unilever
Start_Date     2000-01-03 05:00:00+00:00
End_Date       2025-07-03 04:00:00+00:00
Start_Price                     0.440314
End_Price                      57.123383
Increase                    12873.335937
Name: 57, dtype: object


In [7]:
#question 7
d4={}
print(df["Brand_Name"].unique())
for x in df["Brand_Name"].unique():
    d4[x]=["","",0]
# print(df["Brand_Name"].unique()[0])
mx=0
for i in df["Brand_Name"].unique():
    l=[]
    for x,y in df.iterrows():
        # print(i,y[6])
        if y[6]!=i:
            continue
        l.append([y[0],y[4]])
    # print(sorted(l,key=lambda x: x[1],reverse=True))
    # break
    for x in range(len(l)-5):
        a=l[x][0]
        b=l[x+5][0]
        cnt=0
        for y in range(5):
            cnt+=l[x+y][1]
        # print(d4[i][2])
        mx=max(mx,cnt)
        if d4[i][2]<=cnt:
            d4[i][2]=cnt
            d4[i][0]=a
            d4[i][1]=b
    # print(mx)
    # break
print(d4)

['peloton' 'crocs' 'adidas' 'amazon' 'apple' 'nike' 'target' 'google'
 'spotify' 'zoom video communications' 'the walt disney company' 'roblox'
 'delta air lines' 'costco' 'southwest airlines'
 'american eagle outfitters' 'tesla' 'starbucks' 'nvidia'
 'salesforce / slack' 'honda' 'colgate palmolive' 'hershey company'
 'chipotle' 'pinterest' 'logitech' 'shopify' 'amd' 'american express'
 'coinbase' 'mastercard' "mcdonald's" 'adobe' 'unilever' 'cisco'
 'jpmorgan chase & co' 'airbnb' 'marriott' 'toyota' 'hilton'
 'the home depot' 'johnson & johnson' 'uber' 'procter & gamble' 'fedex'
 '3m' 'philips' 'foot locker' 'the coca-cola company' 'microsoft' 'visa'
 'lvmh' 'zoominfo' 'ubisoft' 'netflix' 'puma' 'nintendo' 'bmw group'
 'porsche' 'twitter' 'nordstrom' 'block']


C:\Users\ryanl\AppData\Local\Temp\ipykernel_14912\748763019.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if y[6]!=i:
C:\Users\ryanl\AppData\Local\Temp\ipykernel_14912\748763019.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  l.append([y[0],y[4]])
C:\Users\ryanl\AppData\Local\Temp\ipykernel_14912\748763019.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if y[6]!=i:
C:\Users\ryanl\AppData\Local\Temp\ipykernel_14912\7

KeyboardInterrupt: 